# Hyperopt



In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import multiprocessing
from sklearn.metrics import roc_auc_score

In [ ]:
## we start again by reading our data

eye_movements = pd.read_csv("../data/eye_movements_aggregated.csv")
eye_movements.head()

In [ ]:
eye_movements.shape

In [ ]:
eye_features = eye_movements.loc[:,['fixcount', 'firstPassCnt', 'P1stFixation', 'P2stFixation',
       'prevFixDur', 'firstfixDur', 'firstPassFixDur', 'nextFixDur',
       'firstSaccLen', 'lastSaccLen', 'prevFixPos', 'landingPos', 'leavingPos',
       'totalFixDur', 'meanFixDur', 'nRegressFrom', 'regressLen',
       'nextWordRegress', 'regressDur', 'pupilDiamMax', 'pupilDiamLag',
       'timePrtctg']]
labels = eye_movements['target'].astype(int)

In [ ]:
##let's start by splitting the data into a train and a validation set
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(eye_features, labels, test_size=0.2, random_state=123)


we train a model to gain an idea of baseline performance 

In [ ]:
%%time 
import xgboost as xgb
xgb_model = xgb.XGBClassifier(n_jobs=multiprocessing.cpu_count()-2)

xgb_model.fit(X_train, y_train)

In [ ]:
print( 'train st performance:',roc_auc_score(y_train , xgb_model.predict_proba( X_train ) , multi_class='ovr' ) )
print( 'valid set performance:',roc_auc_score(y_valid , xgb_model.predict_proba( X_valid ) , multi_class='ovr') )

# "basic" usage 

In [ ]:
from hyperopt import hp, fmin, tpe, rand, pyll, STATUS_OK, STATUS_FAIL, Trials, space_eval

## function to optimize

We need to write a function which :
 * takes a dictionary of parameters as arguments
 * returns a dictionnary containing the computed loss and a status value
 
Importantly, the loss is a value which we want to **minimize**, so in our case we will
use -AUC(validation data)

In [ ]:

def hyperopt_xgb_train( params ):
    
    xgb_model = xgb.XGBClassifier(n_jobs=multiprocessing.cpu_count()-2 , **params)
    xgb_model.fit(X_train, y_train)
    
    loss = -roc_auc_score(y_valid , xgb_model.predict_proba( X_valid ) , multi_class='ovr' )
    return_dict = {'loss': loss,
                   'status': STATUS_OK
                   }
    return return_dict


In [ ]:
params = dict( n_estimators=100,
               eta = 0.3,
               subsample = 0.8,
               max_depth = 5)
hyperopt_xgb_train( params )

## parameter search-space

The way it works is that for each hyper-parameter you define a **prior distribution** (it is bayesian afterall) using one of hyperopt function.

For a more detailed documentation we refer you to the [hyperopt documentation](http://hyperopt.github.io/hyperopt/getting-started/search_spaces/)

In [ ]:
simple_uniform = hp.uniform('simple', 0, 1)
simple_uniform

In [ ]:
pyll.stochastic.sample(simple_uniform)

In [ ]:
eta = hp.loguniform('eta' , np.log(10**-4) , np.log(10**2) )
pyll.stochastic.sample(eta)

In [ ]:
sns.histplot( [ np.log10( pyll.stochastic.sample(eta) ) for _ in range(10000) ] )

**exercise** : which prior distribution could we give `n_estimators`? to `max_depth` ? 

In [ ]:
# %load solutions/solution_hyperopt_parameter_space.py

## minimizing

In [ ]:
%%time
trials = Trials()

best = fmin(hyperopt_xgb_train, 
            space4xgb, 
            algo=tpe.suggest, 
            max_evals=20, 
            trials=trials)

In [ ]:
best

In [ ]:
len( trials )

In [ ]:
trials.trials

In [ ]:
trials.results[:10]

In [ ]:
trials.losses()[:10]

In [ ]:
trials.vals.keys()

In [ ]:
for k in trials.vals.keys():
    print(k , trials.vals[k][:5])

In [ ]:
fig,ax = plt.subplots(figsize=(10,5))

sns.kdeplot( np.log10( trials.vals['eta'] ) , ax=ax , cut = 0 , label='sampled')
sns.histplot( np.log10( trials.vals['eta'] ) , binwidth=0.5 , ax=ax , label='sampled' , stat = 'density')

sns.kdeplot( [ np.log10( pyll.stochastic.sample(eta) ) for _ in range(10000) ] , ax=ax , cut = 0 , label='prior')
ax.set_xlabel('eta')
ax.legend()

In [ ]:

ax = sns.scatterplot( x = np.log10( trials.vals['eta'] ) ,
                 y =  trials.losses() )
ax.set_xlabel('eta')
ax.set_ylabel('loss')

In [ ]:
ax = sns.scatterplot( x =  range(len(trials.losses())) ,  y =  trials.losses() )
ax.set_xlabel('trials')
ax.set_ylabel('loss')

# changing the cross-validation scheme

In [ ]:
from sklearn.model_selection import StratifiedKFold

In [ ]:
skf = StratifiedKFold(n_splits=5 , shuffle=True , random_state = 2024 )
for t,v in skf.split( eye_features, labels ):

    print(len(t), t[:5])
    print(len(v), v[:5])

In [ ]:
from sklearn.model_selection import cross_val_score
cross_val_score( xgb.XGBClassifier(n_jobs=multiprocessing.cpu_count()-2) , 
                eye_features, labels, scoring = 'roc_auc_ovr' , cv = skf )

### exercise

Write the function we will give to `fmin` to evaluate an XGBoost classifier with 5-fold CV with a ROC AUC score.


In [ ]:
def hyperopt_xgb_train_CV5( params ):
    ...
    

In [ ]:
# %load solutions/solution_hyperopt_CV.py

In [ ]:
## use this code to test your function:
params = dict( n_estimators=100,
               eta = 0.3,
               subsample = 0.8,
               max_depth = 5)
hyperopt_xgb_train_CV5( params )

In [ ]:
%%time
trials = Trials()

best = fmin(hyperopt_xgb_train_CV5, 
            space4xgb, 
            algo=tpe.suggest, 
            max_evals=10, 
            trials=trials)

In [ ]:
len( trials )

# warm start : continue searching for better solutions

In [ ]:
best = fmin(hyperopt_xgb_train_CV5, 
            space4xgb, 
            algo=tpe.suggest, 
            max_evals=14, 
            trials=trials)

In [ ]:
len(trials)

# nested search space

It is possible nest elements in `hp.choice()` or `hp.pchoice()` to specify more complex search spaces:

In [ ]:
space4nested = hp.pchoice('classifier_type',
                          [(0.6, {'classifier_type': 'xgboost',
                                    'n_estimators': pyll.scope.int(hp.quniform('n_estimators', 1 , 1000,1)),
                                    'eta': hp.loguniform('eta' , np.log(10**-4) , np.log(10**2) ),
                                    'max_depth': pyll.scope.int(hp.quniform('max_depth', 1, 16, 1)),
                                    'subsample': hp.uniform('subsample', 0.3, 1)
                                  }
                            ),
                           (0.4, {'classifier_type': 'logistic regression',
                                  'C': hp.loguniform("C", low=np.log(1e-5), high=np.log(1e2)),
                                  'penalty': hp.choice('penalty', ['l1','l2'])
                                  }
                            )]
                          )

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


def hyperopt_complex_train_CV5( params ):
    
    classif_type = params.pop('classifier_type')
    
    model = None
    if classif_type == 'xgboost':
        model = xgb.XGBClassifier(n_jobs=multiprocessing.cpu_count()-2 , **params) 
    
    else:
        model =Pipeline([('scalar',StandardScaler()), 
                      ('model',LogisticRegression(solver ='liblinear', class_weight='balanced', **params))])
    
    losses = -1 * cross_val_score( model  , 
                    eye_features, labels, scoring = 'accuracy' , cv = skf )
        
    return_dict = {'loss': np.mean(losses),
                   'loss_variance' : np.var(losses),
                   'status': STATUS_OK
                   }
    return return_dict


In [ ]:
%%time
trials = Trials()
best = fmin(hyperopt_complex_train_CV5, 
            space4nested, 
            algo=tpe.suggest, 
            max_evals=15, 
            trials=trials)

In [ ]:
trials.trials

In [ ]:
ax = sns.scatterplot( x =  range(len(trials.losses())) ,  y =  trials.losses() , 
                     hue = np.array(['xgboost','logistic regression'])[trials.vals['classifier_type']] )
ax.set_xlabel('trials')
ax.set_ylabel('loss')